In [1]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Any

# Statistical modeling
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ All imports successful!")
print(f"📊 Pandas version: {pd.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print("🚀 Ready to start rolling forecast experiment!")

✅ All imports successful!
📊 Pandas version: 2.2.3
📈 Matplotlib version: 3.7.5
🚀 Ready to start rolling forecast experiment!


In [2]:
# Cell 2: Helper Functions

def naive_forecast(historical_data: pd.Series, steps_ahead: int = 1) -> float:
    """
    Naive benchmark: yesterday's price = today's prediction
    
    Args:
        historical_data: Time series of historical prices
        steps_ahead: Number of steps to forecast (default=1 for next hour)
    
    Returns:
        Forecasted value (last observed value)
    """
    if len(historical_data) == 0:
        return np.nan
    return historical_data.iloc[-1]


def fit_sarima_model(data: pd.Series, order: Tuple[int, int, int], 
                     seasonal_order: Tuple[int, int, int, int]) -> Optional[Any]:
    """
    Fit SARIMA model with error handling
    
    Args:
        data: Time series data
        order: (p, d, q) parameters
        seasonal_order: (P, D, Q, s) parameters
    
    Returns:
        Fitted SARIMAX model or None if fitting fails
    """
    try:
        model = SARIMAX(data, 
                       order=order, 
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False)
        
        fitted_model = model.fit(disp=False, maxiter=100)
        return fitted_model
    
    except Exception as e:
        print(f"⚠️ SARIMA fitting failed: {str(e)[:100]}...")
        return None


def fit_sarimax_model(endog: pd.Series, exog: pd.DataFrame, 
                      order: Tuple[int, int, int], 
                      seasonal_order: Tuple[int, int, int, int]) -> Optional[Any]:
    """
    Fit SARIMAX model with exogenous variables
    
    Args:
        endog: Target time series
        exog: Exogenous variables DataFrame
        order: (p, d, q) parameters
        seasonal_order: (P, D, Q, s) parameters
    
    Returns:
        Fitted SARIMAX model or None if fitting fails
    """
    try:
        model = SARIMAX(endog, 
                       exog=exog,
                       order=order, 
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False)
        
        fitted_model = model.fit(disp=False, maxiter=100)
        return fitted_model
    
    except Exception as e:
        print(f"⚠️ SARIMAX fitting failed: {str(e)[:100]}...")
        return None


def calculate_rmse(actual: float, predicted: float) -> float:
    """
    Calculate Root Mean Square Error for single point
    
    Args:
        actual: Actual observed value
        predicted: Predicted value
    
    Returns:
        RMSE value
    """
    if pd.isna(actual) or pd.isna(predicted):
        return np.nan
    return np.sqrt((actual - predicted) ** 2)


def validate_training_data(df: pd.DataFrame) -> bool:
    """
    Validate the structure of training_data DataFrame
    
    Args:
        df: Training data DataFrame
    
    Returns:
        True if validation passes, False otherwise
    """
    required_cols = ['Price']
    
    # Check if required columns exist
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"❌ Missing required columns: {missing_cols}")
        return False
    
    # Check if index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        print("❌ Index must be DatetimeIndex")
        return False
    
    # Check for missing values in Price column
    if df['Price'].isna().any():
        print("❌ Price column contains NaN values")
        return False
    
    print("✅ Training data validation passed!")
    return True

print("✅ Helper functions defined successfully!")

✅ Helper functions defined successfully!


In [3]:
# Cell 3: Main Rolling Forecast Class

class RollingForecastExperiment:
    """
    Implements 30-day rolling forecast experiment for electricity price forecasting
    """
    
    def __init__(self, training_data: pd.DataFrame, 
                 sarima_order: Tuple[int, int, int] = (1, 1, 1),
                 seasonal_order: Tuple[int, int, int, int] = (1, 1, 1, 24),
                 exog_variables: List[str] = None):
        """
        Initialize the rolling forecast experiment
        
        Args:
            training_data: DataFrame with Price and exogenous variables
            sarima_order: (p, d, q) parameters for SARIMA
            seasonal_order: (P, D, Q, s) parameters for seasonal component
            exog_variables: List of exogenous variable column names
        """
        self.data = training_data.copy()
        self.sarima_order = sarima_order
        self.seasonal_order = seasonal_order
        self.exog_variables = exog_variables or []
        
        # Results storage
        self.results = {
            'dates': [],
            'actual_prices': [],
            'naive_forecasts': [],
            'sarima_forecasts': [],
            'sarimax_forecasts': [],
            'naive_rmse': [],
            'sarima_rmse': [],
            'sarimax_rmse': []
        }
        
        # Model storage for analysis
        self.fitted_models = {
            'sarima': [],
            'sarimax': []
        }
        
        print(f"🔧 Experiment initialized:")
        print(f"   - SARIMA order: {sarima_order}")
        print(f"   - Seasonal order: {seasonal_order}")
        print(f"   - Exogenous variables: {len(self.exog_variables)}")
    
    
    def get_actual_data_end_date(self) -> pd.Timestamp:
        """
        Find the last date with actual price data (not forecasted)
        Assumes forecasted data has been appended to the end
        """
        # This is a heuristic - you might need to adjust based on your data structure
        # Look for the last non-interpolated or non-repeated price pattern
        price_series = self.data['Price']
        
        # Simple approach: assume actual data is complete up to a certain point
        # You may need to modify this based on your specific data structure
        return price_series.index[-169]  # Assuming last 168 hours are forecasted
    
    
    def run_experiment(self, start_date: str, num_days: int = 30) -> Dict:
        """
        Run the rolling forecast experiment
        
        Args:
            start_date: Start date for the experiment (YYYY-MM-DD)
            num_days: Number of days to forecast
        
        Returns:
            Dictionary with experiment results
        """
        start_dt = pd.to_datetime(start_date)
        
        print(f"🚀 Starting {num_days}-day rolling forecast experiment from {start_date}")
        print("=" * 60)
        
        for day in range(num_days):
            forecast_date = start_dt + timedelta(days=day)
            
            print(f"📅 Day {day + 1}/{num_days}: Forecasting for {forecast_date.strftime('%Y-%m-%d')}")
            
            # Get training data up to (but not including) forecast date
            train_end = forecast_date - timedelta(hours=1)
            training_subset = self.data.loc[:train_end]
            
            if len(training_subset) < 168:  # Need at least 1 week of data
                print(f"   ⚠️ Insufficient training data, skipping...")
                continue
            
            # Get actual value for forecast date (first hour of the day)
            forecast_timestamp = forecast_date.replace(hour=0, minute=0, second=0)
            
            if forecast_timestamp not in self.data.index:
                print(f"   ⚠️ No actual data available for {forecast_timestamp}, skipping...")
                continue
                
            actual_price = self.data.loc[forecast_timestamp, 'Price']
            
            # Generate forecasts from each model
            forecasts = self._generate_forecasts(training_subset, forecast_timestamp)
            
            # Calculate RMSEs
            rmse_naive = calculate_rmse(actual_price, forecasts['naive'])
            rmse_sarima = calculate_rmse(actual_price, forecasts['sarima'])
            rmse_sarimax = calculate_rmse(actual_price, forecasts['sarimax'])
            
            # Store results
            self.results['dates'].append(forecast_timestamp)
            self.results['actual_prices'].append(actual_price)
            self.results['naive_forecasts'].append(forecasts['naive'])
            self.results['sarima_forecasts'].append(forecasts['sarima'])
            self.results['sarimax_forecasts'].append(forecasts['sarimax'])
            self.results['naive_rmse'].append(rmse_naive)
            self.results['sarima_rmse'].append(rmse_sarima)
            self.results['sarimax_rmse'].append(rmse_sarimax)
            
            # Print daily results
            print(f"   💰 Actual: {actual_price:.2f}")
            print(f"   🔮 Naive: {forecasts['naive']:.2f} (RMSE: {rmse_naive:.2f})")
            print(f"   📊 SARIMA: {forecasts['sarima']:.2f} (RMSE: {rmse_sarima:.2f})")
            print(f"   🎯 SARIMAX: {forecasts['sarimax']:.2f} (RMSE: {rmse_sarimax:.2f})")
            print("-" * 40)
        
        print("✅ Rolling forecast experiment completed!")
        return self._calculate_summary_stats()
    
    
    def _generate_forecasts(self, training_data: pd.DataFrame, 
                          forecast_timestamp: pd.Timestamp) -> Dict[str, float]:
        """
        Generate forecasts from all three models
        """
        forecasts = {'naive': np.nan, 'sarima': np.nan, 'sarimax': np.nan}
        
        price_series = training_data['Price']
        
        # 1. Naive forecast
        forecasts['naive'] = naive_forecast(price_series)
        
        # 2. SARIMA forecast
        sarima_model = fit_sarima_model(price_series, self.sarima_order, self.seasonal_order)
        if sarima_model is not None:
            try:
                sarima_pred = sarima_model.forecast(steps=1)
                forecasts['sarima'] = sarima_pred.iloc[0] if hasattr(sarima_pred, 'iloc') else sarima_pred[0]
                self.fitted_models['sarima'].append(sarima_model)
            except Exception as e:
                print(f"   ⚠️ SARIMA forecast failed: {str(e)[:50]}...")
        
        # 3. SARIMAX forecast
        if self.exog_variables:
            try:
                exog_train = training_data[self.exog_variables]
                
                # Get exogenous data for forecast point
                if forecast_timestamp in self.data.index:
                    exog_forecast = self.data.loc[[forecast_timestamp], self.exog_variables]
                    
                    sarimax_model = fit_sarimax_model(price_series, exog_train, 
                                                    self.sarima_order, self.seasonal_order)
                    if sarimax_model is not None:
                        sarimax_pred = sarimax_model.forecast(steps=1, exog=exog_forecast)
                        forecasts['sarimax'] = sarimax_pred.iloc[0] if hasattr(sarimax_pred, 'iloc') else sarimax_pred[0]
                        self.fitted_models['sarimax'].append(sarimax_model)
                        
            except Exception as e:
                print(f"   ⚠️ SARIMAX forecast failed: {str(e)[:50]}...")
        
        return forecasts
    
    
    def _calculate_summary_stats(self) -> Dict:
        """
        Calculate summary statistics for the experiment
        """
        summary = {}
        
        for model in ['naive', 'sarima', 'sarimax']:
            rmse_values = [x for x in self.results[f'{model}_rmse'] if not pd.isna(x)]
            
            if rmse_values:
                summary[model] = {
                    'mean_rmse': np.mean(rmse_values),
                    'std_rmse': np.std(rmse_values),
                    'min_rmse': np.min(rmse_values),
                    'max_rmse': np.max(rmse_values),
                    'successful_forecasts': len(rmse_values)
                }
            else:
                summary[model] = {
                    'mean_rmse': np.nan,
                    'std_rmse': np.nan,
                    'min_rmse': np.nan,
                    'max_rmse': np.nan,
                    'successful_forecasts': 0
                }
        
        return summary
    
    
    def get_results_dataframe(self) -> pd.DataFrame:
        """
        Return results as a pandas DataFrame
        """
        return pd.DataFrame(self.results)

print("✅ RollingForecastExperiment class defined successfully!")

✅ RollingForecastExperiment class defined successfully!


In [4]:
# Cell 4: Configuration and Data Preparation

# =============================================================================
# EXPERIMENT CONFIGURATION
# =============================================================================

# Model parameters
SARIMA_ORDER = (1, 1, 1)  # (p, d, q) - adjust based on your data characteristics
SEASONAL_ORDER = (1, 1, 1, 24)  # (P, D, Q, s) - 24 for hourly seasonality

# Experiment parameters
EXPERIMENT_START_DATE = "2024-03-15"  # Adjust to your preferred start date
NUM_FORECAST_DAYS = 30

# Exogenous variables (adjust based on your available columns)
EXOG_VARIABLES = [
'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 
    'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 
    'yearday_sin', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 
    'weekday_sin', 'hour_sin', 'weekday_cos'
]

print("🔧 EXPERIMENT CONFIGURATION")
print("=" * 50)
print(f"📅 Start Date: {EXPERIMENT_START_DATE}")
print(f"⏰ Forecast Days: {NUM_FORECAST_DAYS}")
print(f"📊 SARIMA Order: {SARIMA_ORDER}")
print(f"🔄 Seasonal Order: {SEASONAL_ORDER}")
print(f"📈 Exogenous Variables: {len(EXOG_VARIABLES)}")
print("\n📋 Exogenous Variables List:")
for i, var in enumerate(EXOG_VARIABLES, 1):
    print(f"   {i}. {var}")

# =============================================================================
# DATA VALIDATION
# =============================================================================

print("\n🔍 VALIDATING TRAINING DATA")
print("=" * 50)

# Check if training_data exists
try:
    print(f"📊 Dataset shape: {training_data.shape}")
    print(f"📅 Date range: {training_data.index.min()} to {training_data.index.max()}")
    print(f"💰 Price column stats:")
    print(f"   - Mean: {training_data['Price'].mean():.2f}")
    print(f"   - Std: {training_data['Price'].std():.2f}")
    print(f"   - Min: {training_data['Price'].min():.2f}")
    print(f"   - Max: {training_data['Price'].max():.2f}")
    
    # Validate data structure
    is_valid = validate_training_data(training_data)
    
    if is_valid:
        # Check availability of exogenous variables
        available_exog = [var for var in EXOG_VARIABLES if var in training_data.columns]
        missing_exog = [var for var in EXOG_VARIABLES if var not in training_data.columns]
        
        if missing_exog:
            print(f"⚠️ Missing exogenous variables: {missing_exog}")
            print(f"✅ Available exogenous variables: {available_exog}")
            EXOG_VARIABLES = available_exog  # Update to only use available variables
            print(f"🔄 Updated exogenous variables list to {len(EXOG_VARIABLES)} variables")
        
        # Check data coverage for experiment period
        experiment_start = pd.to_datetime(EXPERIMENT_START_DATE)
        experiment_end = experiment_start + timedelta(days=NUM_FORECAST_DAYS)
        
        data_start = training_data.index.min()
        data_end = training_data.index.max()
        
        print(f"\n📅 Data Coverage Check:")
        print(f"   - Experiment needs: {experiment_start} to {experiment_end}")
        print(f"   - Data available: {data_start} to {data_end}")
        
        if experiment_start < data_start:
            print("❌ Experiment start date is before available data")
        elif experiment_end > data_end:
            print("⚠️ Experiment end date extends beyond available data")
            print("   This is expected if you have forecasted exogenous variables")
        else:
            print("✅ Data coverage is sufficient for the experiment")
        
        print("\n✅ Data validation completed successfully!")
        
    else:
        print("❌ Data validation failed. Please check your training_data DataFrame.")
        
except NameError:
    print("❌ training_data DataFrame not found!")
    print("Please ensure your training_data DataFrame is loaded before running this cell.")
except Exception as e:
    print(f"❌ Error during data validation: {str(e)}")

print("\n🚀 Ready to run the rolling forecast experiment!")

🔧 EXPERIMENT CONFIGURATION
📅 Start Date: 2024-03-15
⏰ Forecast Days: 30
📊 SARIMA Order: (1, 1, 1)
🔄 Seasonal Order: (1, 1, 1, 24)
📈 Exogenous Variables: 18

📋 Exogenous Variables List:
   1. Load
   2. shortwave_radiation
   3. temperature_2m
   4. direct_normal_irradiance
   5. diffuse_radiation
   6. Flow_NO
   7. yearday_cos
   8. Flow_GB
   9. month
   10. is_dst
   11. yearday_sin
   12. is_non_working_day
   13. hour_cos
   14. is_weekend
   15. cloud_cover
   16. weekday_sin
   17. hour_sin
   18. weekday_cos

🔍 VALIDATING TRAINING DATA
❌ training_data DataFrame not found!
Please ensure your training_data DataFrame is loaded before running this cell.

🚀 Ready to run the rolling forecast experiment!


In [5]:
# Cell 5: Execute Rolling Forecast Experiment

# =============================================================================
# INITIALIZE AND RUN EXPERIMENT
# =============================================================================

print("🚀 INITIALIZING ROLLING FORECAST EXPERIMENT")
print("=" * 60)

# Create the experiment instance
experiment = RollingForecastExperiment(
    training_data=training_data,
    sarima_order=SARIMA_ORDER,
    seasonal_order=SEASONAL_ORDER,
    exog_variables=EXOG_VARIABLES
)

print("\n⏳ Running experiment... This may take several minutes.")
print("💡 Tip: Model fitting can be time-consuming, especially for SARIMAX models")

# Record start time
import time
start_time = time.time()

# Run the experiment
try:
    summary_stats = experiment.run_experiment(
        start_date=EXPERIMENT_START_DATE,
        num_days=NUM_FORECAST_DAYS
    )
    
    # Record end time
    end_time = time.time()
    execution_time = end_time - start_time
    
    print(f"\n⏱️ Experiment completed in {execution_time:.1f} seconds ({execution_time/60:.1f} minutes)")
    
    # Display summary statistics
    print("\n📊 EXPERIMENT SUMMARY STATISTICS")
    print("=" * 60)
    
    for model_name, stats in summary_stats.items():
        print(f"\n🔹 {model_name.upper()} MODEL:")
        print(f"   Mean RMSE: {stats['mean_rmse']:.4f}")
        print(f"   Std RMSE:  {stats['std_rmse']:.4f}")
        print(f"   Min RMSE:  {stats['min_rmse']:.4f}")
        print(f"   Max RMSE:  {stats['max_rmse']:.4f}")
        print(f"   Success Rate: {stats['successful_forecasts']}/{NUM_FORECAST_DAYS}")
    
    # Store results for further analysis
    results_df = experiment.get_results_dataframe()
    
    print("\n✅ Experimental results stored in 'results_df' DataFrame")
    print("✅ Summary statistics stored in 'summary_stats' dictionary")
    print("✅ Experiment object stored in 'experiment' variable")

except Exception as e:
    print(f"\n❌ Experiment failed with error: {str(e)}")
    print("🔧 This might be due to:")
    print("   - Insufficient historical data")
    print("   - Model convergence issues")
    print("   - Missing exogenous variables")
    print("   - Date range issues")
    
    # Still try to get partial results
    try:
        results_df = experiment.get_results_dataframe()
        if len(results_df) > 0:
            print(f"\n📊 Partial results available: {len(results_df)} forecasts completed")
        else:
            print("\n📊 No results available")
    except:
        print("\n📊 No results could be retrieved")

print("\n🎯 Ready for results analysis and visualization!")

🚀 INITIALIZING ROLLING FORECAST EXPERIMENT


NameError: name 'training_data' is not defined

In [ ]:
# Cell 6: Results Analysis and Visualization

# =============================================================================
# DETAILED RESULTS ANALYSIS
# =============================================================================

print("📊 DETAILED RESULTS ANALYSIS")
print("=" * 60)

try:
    # Display results dataframe summary
    print(f"📋 Results DataFrame Shape: {results_df.shape}")
    print(f"📅 Date Range: {results_df['dates'].min()} to {results_df['dates'].max()}")
    
    # Calculate additional metrics
    valid_results = results_df.dropna(subset=['actual_prices'])
    
    if len(valid_results) > 0:
        print(f"\n✅ Valid forecasts: {len(valid_results)}/{len(results_df)}")
        
        # Price statistics
        print(f"\n💰 ACTUAL PRICE STATISTICS:")
        print(f"   Mean: {valid_results['actual_prices'].mean():.2f}")
        print(f"   Std:  {valid_results['actual_prices'].std():.2f}")
        print(f"   Min:  {valid_results['actual_prices'].min():.2f}")
        print(f"   Max:  {valid_results['actual_prices'].max():.2f}")
        
        # Model comparison table
        print(f"\n📊 MODEL COMPARISON TABLE:")
        print("-" * 80)
        print(f"{'Model':<12} {'Mean RMSE':<12} {'Std RMSE':<12} {'Min RMSE':<12} {'Max RMSE':<12}")
        print("-" * 80)
        
        for model in ['naive', 'sarima', 'sarimax']:
            if model in summary_stats:
                stats = summary_stats[model]
                print(f"{model.capitalize():<12} "
                      f"{stats['mean_rmse']:<12.4f} "
                      f"{stats['std_rmse']:<12.4f} "
                      f"{stats['min_rmse']:<12.4f} "
                      f"{stats['max_rmse']:<12.4f}")
        print("-" * 80)

except NameError:
    print("❌ Results not available. Please run the experiment first (Cell 5).")
    print("🔄 Attempting to create sample visualization with dummy data...")

# =============================================================================
# VISUALIZATIONS
# =============================================================================

print("\n📈 CREATING VISUALIZATIONS")
print("=" * 60)

try:
    # Create a comprehensive visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('30-Day Rolling Forecast Experiment Results', fontsize=16, fontweight='bold')
    
    # 1. RMSE Evolution Over Time
    ax1 = axes[0, 0]
    valid_results = results_df.dropna(subset=['dates'])
    
    if len(valid_results) > 0:
        ax1.plot(valid_results['dates'], valid_results['naive_rmse'], 
                marker='o', label='Naive', linewidth=2, markersize=4)
        ax1.plot(valid_results['dates'], valid_results['sarima_rmse'], 
                marker='s', label='SARIMA', linewidth=2, markersize=4)
        ax1.plot(valid_results['dates'], valid_results['sarimax_rmse'], 
                marker='^', label='SARIMAX', linewidth=2, markersize=4)
        
        ax1.set_title('RMSE Evolution Over 30 Days')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('RMSE')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
    
    # 2. Box Plot of RMSE Distribution
    ax2 = axes[0, 1]
    rmse_data = []
    labels = []
    
    for model in ['naive', 'sarima', 'sarimax']:
        rmse_col = f'{model}_rmse'
        if rmse_col in results_df.columns:
            model_rmse = results_df[rmse_col].dropna()
            if len(model_rmse) > 0:
                rmse_data.append(model_rmse)
                labels.append(model.capitalize())
    
    if rmse_data:
        ax2.boxplot(rmse_data, labels=labels)
        ax2.set_title('RMSE Distribution by Model')
        ax2.set_ylabel('RMSE')
        ax2.grid(True, alpha=0.3)
    
    # 3. Actual vs Predicted Scatter Plot
    ax3 = axes[1, 0]
    if len(valid_results) > 0:
        # SARIMAX scatter (best model typically)
        valid_pred = valid_results.dropna(subset=['actual_prices', 'sarimax_forecasts'])
        if len(valid_pred) > 0:
            ax3.scatter(valid_pred['actual_prices'], valid_pred['sarimax_forecasts'], 
                       alpha=0.6, s=50, label='SARIMAX')
            
        # Perfect prediction line
        if len(valid_results) > 0:
            min_price = valid_results['actual_prices'].min()
            max_price = valid_results['actual_prices'].max()
            ax3.plot([min_price, max_price], [min_price, max_price], 
                    'r--', alpha=0.8, label='Perfect Prediction')
        
        ax3.set_title('Actual vs Predicted Prices (SARIMAX)')
        ax3.set_xlabel('Actual Price')
        ax3.set_ylabel('Predicted Price')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
    
    # 4. Model Performance Summary Bar Chart
    ax4 = axes[1, 1]
    models = []
    mean_rmses = []
    
    for model in ['naive', 'sarima', 'sarimax']:
        if model in summary_stats and not pd.isna(summary_stats[model]['mean_rmse']):
            models.append(model.capitalize())
            mean_rmses.append(summary_stats[model]['mean_rmse'])
    
    if models and mean_rmses:
        bars = ax4.bar(models, mean_rmses, 
                      color=['skyblue', 'lightcoral', 'lightgreen'])
        ax4.set_title('Mean RMSE by Model')
        ax4.set_ylabel('Mean RMSE')
        ax4.grid(True, alpha=0.3, axis='y')
        
        # Add value labels on bars
        for bar, value in zip(bars, mean_rmses):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(mean_rmses)*0.01,
                    f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualization completed successfully!")

except Exception as e:
    print(f"⚠️ Visualization error: {str(e)}")
    print("🔧 This might be due to insufficient data or missing results")

# =============================================================================
# FEATURE IMPORTANCE ANALYSIS (SARIMAX)
# =============================================================================

print(f"\n🎯 SARIMAX FEATURE IMPORTANCE ANALYSIS")
print("=" * 60)

try:
    if hasattr(experiment, 'fitted_models') and experiment.fitted_models['sarimax']:
        # Get the last fitted SARIMAX model for analysis
        last_sarimax_model = experiment.fitted_models['sarimax'][-1]
        
        if hasattr(last_sarimax_model, 'params') and len(EXOG_VARIABLES) > 0:
            print("📊 EXOGENOUS VARIABLE COEFFICIENTS (Last Model):")
            print("-" * 50)
            
            # Extract coefficients for exogenous variables
            params = last_sarimax_model.params
            
            # The exogenous coefficients typically come after the ARIMA parameters
            # This is a simplified approach - you might need to adjust based on your model structure
            if len(params) > len(SARIMA_ORDER) + len(SEASONAL_ORDER):
                exog_start_idx = len(SARIMA_ORDER) + len(SEASONAL_ORDER) - 2  # Approximate
                
                for i, var_name in enumerate(EXOG_VARIABLES):
                    if exog_start_idx + i < len(params):
                        coef_value = params.iloc[exog_start_idx + i]
                        print(f"   {var_name:<20}: {coef_value:>8.4f}")
            
            print("\n💡 Note: Coefficient interpretation depends on model specification")
            print("   Larger absolute values indicate stronger influence on price forecasts")
        
        else:
            print("⚠️ Feature importance analysis not available")
            print("   Model parameters not accessible or no exogenous variables")
    
    else:
        print("⚠️ No SARIMAX models available for feature importance analysis")

except Exception as e:
    print(f"⚠️ Feature importance analysis failed: {str(e)}")
    print("🔧 This is normal if SARIMAX models failed to fit")

print(f"\n📋 RESULTS SUMMARY SAVED")
print("=" * 60)
print("✅ All analysis completed!")
print("📊 Key variables available for further analysis:")
print("   - results_df: Detailed daily results")
print("   - summary_stats: Model performance summary")
print("   - experiment: Full experiment object with fitted models")

In [ ]:
# Cell 7: Model Recommendation and Insights

# =============================================================================
# MODEL RECOMMENDATION ENGINE
# =============================================================================

print("🎯 MODEL RECOMMENDATION AND INSIGHTS")
print("=" * 60)

def get_best_model_recommendation(summary_stats: Dict) -> Dict:
    """
    Determine the best performing model based on multiple criteria
    """
    recommendation = {
        'best_model': None,
        'best_rmse': float('inf'),
        'criteria': {},
        'insights': []
    }
    
    valid_models = {}
    
    # Filter out models with invalid results
    for model_name, stats in summary_stats.items():
        if not pd.isna(stats['mean_rmse']) and stats['successful_forecasts'] > 0:
            valid_models[model_name] = stats
    
    if not valid_models:
        recommendation['insights'].append("❌ No valid model results found")
        return recommendation
    
    # 1. Best Mean RMSE
    best_rmse_model = min(valid_models.keys(), 
                         key=lambda x: valid_models[x]['mean_rmse'])
    recommendation['best_model'] = best_rmse_model
    recommendation['best_rmse'] = valid_models[best_rmse_model]['mean_rmse']
    
    # 2. Most Consistent (Lowest Std RMSE)
    most_consistent_model = min(valid_models.keys(), 
                               key=lambda x: valid_models[x]['std_rmse'])
    
    # 3. Most Reliable (Highest Success Rate)
    most_reliable_model = max(valid_models.keys(), 
                             key=lambda x: valid_models[x]['successful_forecasts'])
    
    # Store criteria results
    recommendation['criteria'] = {
        'best_accuracy': best_rmse_model,
        'most_consistent': most_consistent_model,
        'most_reliable': most_reliable_model
    }
    
    # Generate insights
    recommendation['insights'].append(
        f"🏆 Best Overall Model: {best_rmse_model.upper()} "
        f"(Mean RMSE: {recommendation['best_rmse']:.4f})"
    )
    
    if most_consistent_model != best_rmse_model:
        recommendation['insights'].append(
            f"📊 Most Consistent: {most_consistent_model.upper()} "
            f"(Std RMSE: {valid_models[most_consistent_model]['std_rmse']:.4f})"
        )
    
    if most_reliable_model != best_rmse_model:
        recommendation['insights'].append(
            f"🔧 Most Reliable: {most_reliable_model.upper()} "
            f"({valid_models[most_reliable_model]['successful_forecasts']}/{NUM_FORECAST_DAYS} forecasts)"
        )
    
    # Performance comparison insights
    rmse_values = [stats['mean_rmse'] for stats in valid_models.values()]
    rmse_improvement = (max(rmse_values) - min(rmse_values)) / max(rmse_values) * 100
    
    recommendation['insights'].append(
        f"📈 Performance Improvement: {rmse_improvement:.1f}% "
        f"(Best vs Worst Model)"
    )
    
    return recommendation

# =============================================================================
# GENERATE RECOMMENDATIONS
# =============================================================================

try:
    # Get model recommendation
    recommendation = get_best_model_recommendation(summary_stats)
    
    print("🏆 MODEL PERFORMANCE RANKING")
    print("-" * 40)
    
    # Rank models by mean RMSE
    valid_models = {name: stats for name, stats in summary_stats.items() 
                   if not pd.isna(stats['mean_rmse'])}
    
    if valid_models:
        sorted_models = sorted(valid_models.items(), 
                              key=lambda x: x[1]['mean_rmse'])
        
        for rank, (model_name, stats) in enumerate(sorted_models, 1):
            medal = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉"
            print(f"{medal} {rank}. {model_name.upper()}")
            print(f"     Mean RMSE: {stats['mean_rmse']:.4f}")
            print(f"     Consistency: {stats['std_rmse']:.4f}")
            print(f"     Success Rate: {stats['successful_forecasts']}/{NUM_FORECAST_DAYS}")
            print()
    
    print("💡 KEY INSIGHTS")
    print("-" * 40)
    for insight in recommendation['insights']:
        print(f"   {insight}")
    
    print(f"\n🎯 FINAL RECOMMENDATION")
    print("-" * 40)
    if recommendation['best_model']:
        print(f"✅ Recommended Model: {recommendation['best_model'].upper()}")
        print(f"📊 Expected RMSE: {recommendation['best_rmse']:.4f}")
        
        # Model-specific recommendations
        if recommendation['best_model'] == 'naive':
            print(f"\n🔍 NAIVE MODEL INSIGHTS:")
            print(f"   • Simple yesterday=today approach works surprisingly well")
            print(f"   • Consider this as a strong baseline for comparison")
            print(f"   • Low computational cost, high interpretability")
            
        elif recommendation['best_model'] == 'sarima':
            print(f"\n🔍 SARIMA MODEL INSIGHTS:")
            print(f"   • Pure time series approach captures patterns effectively")
            print(f"   • Model order: {SARIMA_ORDER} with seasonality {SEASONAL_ORDER}")
            print(f"   • Consider tuning hyperparameters for better performance")
            
        elif recommendation['best_model'] == 'sarimax':
            print(f"\n🔍 SARIMAX MODEL INSIGHTS:")
            print(f"   • Exogenous variables provide valuable additional information")
            print(f"   • Using {len(EXOG_VARIABLES)} external features")
            print(f"   • Most complex model - ensure robust exogenous forecasts")
        
        print(f"\n📋 IMPLEMENTATION CONSIDERATIONS:")
        print(f"   • Computational Cost: {'Low' if recommendation['best_model'] == 'naive' else 'Medium' if recommendation['best_model'] == 'sarima' else 'High'}")
        print(f"   • Data Requirements: {'Minimal' if recommendation['best_model'] == 'naive' else 'Historical Prices' if recommendation['best_model'] == 'sarima' else 'Prices + Exogenous'}")
        print(f"   • Interpretability: {'High' if recommendation['best_model'] == 'naive' else 'Medium'}")
        
    else:
        print("❌ No clear recommendation - all models failed")
        print("🔧 Consider:")
        print("   • Checking data quality and completeness")
        print("   • Adjusting model parameters")
        print("   • Using a longer training period")
    
except NameError:
    print("❌ Summary statistics not available")
    print("🔄 Please run the experiment first (Cell 5)")
except Exception as e:
    print(f"⚠️ Recommendation generation failed: {str(e)}")

# =============================================================================
# ADDITIONAL INSIGHTS AND NEXT STEPS
# =============================================================================

print(f"\n🚀 NEXT STEPS AND IMPROVEMENTS")
print("=" * 60)
print("🎨 Model Enhancement Ideas:")
print("   1. Hyperparameter Tuning:")
print("      • Grid search for optimal SARIMA orders")
print("      • Cross-validation for model selection")
print("   2. Feature Engineering:")
print("      • Lag features, rolling averages")
print("      • Calendar effects (holidays, seasons)")
print("   3. Advanced Models:")
print("      • Machine Learning approaches (XGBoost, LSTM)")
print("      • Ensemble methods combining multiple forecasts")
print("   4. Evaluation Improvements:")
print("      • Multi-step ahead forecasting")
print("      • Additional metrics (MAE, MAPE)")
print("      • Directional accuracy assessment")

print(f"\n📊 Experiment Variables Available:")
print("   • results_df: Daily forecast results")
print("   • summary_stats: Model performance statistics") 
print("   • experiment: Complete experiment object")
print("   • recommendation: Model recommendation analysis")

print(f"\n✅ 30-Day Rolling Forecast Experiment Complete!")
print("🎯 Use the insights above to improve your electricity price forecasting system.")